
<div><div class="highlight-default notranslate"><div class="highlight"><pre><span></span>content/
├─ scripts/ (Google Drive Mount)
│  └─ preprocessing/
├─ models/
│  ├─ community/
│  ├─ official/
│  ├─ orbit/
│  ├─ research/
│  └─ ...
└─ workspace/ (Google Drive Mount)
    └─ training_demo/
</pre></div>


<div><div class="highlight-default notranslate"><div class="highlight"><pre><span></span>training_demo/
├─ annotations/
├─ exported-models/
├─ images/
│  ├─ test/
│  └─ train/
├─ models/
├─ pre-trained-models/
└─ README.md
</pre></div>

In [ ]:
from google.colab import drive
drive.mount('/content/google_drive')

Mounted at /content/google_drive


In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/scripts"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/scripts/preprocessing"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/exported-models"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/test"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models"

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/pre-trained-models"

In [ ]:
!pwd

/content


In [ ]:
!cp "/content/google_drive/MyDrive/tensorflow_OD_API_A/datasets/VOCtrainval_11-May-2012.tar" ./

In [ ]:
!pwd

/content


In [ ]:
!tar -xvf ./VOCtrainval_11-May-2012.tar 

In [ ]:
!ls -al

In [ ]:
imageDir = "/content/VOCdevkit/VOC2012/JPEGImages"

In [ ]:
import os
import pathlib
import time

import matplotlib
import matplotlib.pyplot as plt

import io
import scipy.misc
import numpy as np
from six import BytesIO
from PIL import Image, ImageDraw, ImageFont
from six.moves.urllib.request import urlopen

import tensorflow as tf
# import tensorflow_hub as hub

tf.get_logger().setLevel('ERROR')

In [ ]:
import re
from shutil import copyfile
import argparse
import math
import random

In [ ]:
images = [f for f in os.listdir(imageDir)
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg|.png)$', f)]        # 리눅스 정규식 표현 : 파일에 a-z,A-Z,0-9라 적혀있는 파일이면 모두 가져온다.

In [ ]:
ratio=0.1

In [ ]:
num_images = len(images)
num_test_images = math.ceil(ratio*num_images)
print(num_test_images)

1713


In [ ]:
train_dir = "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train"
test_dir = "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/test"

In [ ]:
source = imageDir

In [ ]:
copy_xml=True

In [ ]:
sourceXML="/content/VOCdevkit/VOC2012/Annotations"

In [ ]:
for i in range(num_test_images):
  idx = random.randint(0, len(images)-1)
  filename = images[idx]
  copyfile(os.path.join(source, filename), os.path.join(test_dir, filename))
  if copy_xml:
    xml_filename = os.path.splitext(filename)[0]+'.xml'
    copyfile(os.path.join(sourceXML, xml_filename), os.path.join(test_dir,xml_filename))

  images.remove(images[idx])

In [ ]:
for filename in images:
  copyfile(os.path.join(source, filename), os.path.join(train_dir, filename))
  if copy_xml:
    xml_filename = os.path.splitext(filename)[0]+'.xml'
    print(copyfile(os.path.join(sourceXML, xml_filename), os.path.join(train_dir, xml_filename)))

In [ ]:
%cd /content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train


In [ ]:
!ls -l | grep ^- | wc -l

30824


In [ ]:
!ls -al

In [ ]:
%cd "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train"

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train


In [ ]:
!pwd

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train


In [ ]:
!nvidia-smi

Mon Nov 21 11:24:49 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  A100-SXM4-40GB      Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    44W / 400W |      0MiB / 40536MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# This Colab requires TF 2.5.
!pip install -U "tensorflow>=2.5"

In [ ]:
!pip install pandas

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')   # Suppress Matplotlib warnings

In [ ]:
!pwd

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train


In [ ]:
%cd /content/

/content


In [ ]:
# Clone the tensorflow models repository
!git clone --depth 1 https://github.com/tensorflow/models

Cloning into 'models'...
remote: Enumerating objects: 3578, done.
remote: Counting objects: 100% (3578/3578), done.
remote: Compressing objects: 100% (2979/2979), done.
remote: Total 3578 (delta 942), reused 1511 (delta 545), pack-reused 0
Receiving objects: 100% (3578/3578), 47.07 MiB | 15.06 MiB/s, done.
Resolving deltas: 100% (942/942), done.


In [ ]:
!sudo apt install -y protobuf-compiler

Reading package lists... Done
Building dependency tree       
Reading state information... Done
protobuf-compiler is already the newest version (3.0.0-9.1ubuntu1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 5 not upgraded.


In [ ]:
%cd /content/models/research

/content/models/research


In [ ]:
!pwd

/content/models/research


In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!git clone https://github.com/cocodataset/cocoapi.git

Cloning into 'cocoapi'...
remote: Enumerating objects: 975, done.
remote: Total 975 (delta 0), reused 0 (delta 0), pack-reused 975
Receiving objects: 100% (975/975), 11.72 MiB | 15.12 MiB/s, done.
Resolving deltas: 100% (576/576), done.


In [ ]:
%cd /content/cocoapi/PythonAPI

/content/cocoapi/PythonAPI


In [ ]:
!ls -al

total 1780
drwxr-xr-x 3 root root    4096 Nov 21 11:33 .
drwxr-xr-x 8 root root    4096 Nov 21 11:33 ..
-rw-r--r-- 1 root root     199 Nov 21 11:33 Makefile
-rw-r--r-- 1 root root 1790308 Nov 21 11:33 pycocoDemo.ipynb
-rw-r--r-- 1 root root    4456 Nov 21 11:33 pycocoEvalDemo.ipynb
drwxr-xr-x 2 root root    4096 Nov 21 11:33 pycocotools
-rw-r--r-- 1 root root     762 Nov 21 11:33 setup.py


In [ ]:
!cp -r pycocotools /content/models/research

In [ ]:
%cd /content

/content


In [ ]:
%cd /content/models/research

/content/models/research


In [ ]:
!ls -al

In [ ]:
!pwd

/content/models/research


In [ ]:
!cp object_detection/packages/tf2/setup.py .

In [ ]:
!python -m pip install .

In [ ]:
!python /content/models/research/object_detection/builders/model_builder_tf2_test.py

In [ ]:
!cp "/content/models/research/object_detection/data/pascal_label_map.pbtxt" \
"/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations"

In [ ]:
!mv "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/pascal_label_map.pbtxt" \
"/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/label_map.pbtxt"

In [ ]:
%cd "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/scripts/preprocessing"

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/scripts/preprocessing


In [ ]:
!pwd

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/scripts/preprocessing


In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

In [ ]:
!python generate_tfrecord.py \
-x "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/train" \
-l "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/label_map.pbtxt" \
-o "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/train.record"

2022-11-21 11:42:46.370307: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Successfully created the TFRecord file: /content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/train.record


In [ ]:
!python generate_tfrecord.py \
-x "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/test" \
-l "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/label_map.pbtxt" \
-o "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/test.record"

2022-11-21 11:44:18.719688: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
Successfully created the TFRecord file: /content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/test.record


In [ ]:
!ls -al "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations"

total 1884544
-rw------- 1 root root        705 Nov 21 11:38 label_map.pbtxt
-rw------- 1 root root  192802550 Nov 21 11:44 test.record
-rw------- 1 root root 1736969035 Nov 21 11:43 train.record


In [ ]:
!pwd

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/scripts/preprocessing


In [ ]:
%cd /content

/content


In [ ]:
!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz

--2022-11-21 12:33:21--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.4.128, 2404:6800:4003:c06::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.4.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 244817203 (233M) [application/x-tar]
Saving to: ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz.1’

ssd_resnet50_v1_fpn 100%[===================>] 233.48M   510MB/s    in 0.5s    

2022-11-21 12:33:21 (510 MB/s) - ‘ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz.1’ saved [244817203/244817203]



In [ ]:
!tar xvfzp "/content/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz" \
-C "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/pre-trained-models"

ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn"

In [ ]:
!cp "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/pipeline.config" \
"/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn" \
--pipeline_config_path="/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn/pipeline.config"

**모델 저장**

In [ ]:
!mkdir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/exported-models/saved_faster_rcnn_resnet50_v1"

In [ ]:
!python /content/models/research/object_detection/exporter_main_v2.py \
--input_type image_tensor \
--pipeline_config_path "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn/pipeline.config" \
--trained_checkpoint_dir "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn" \
--output_directory "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/exported-models/saved_faster_rcnn_resnet50_v1"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn" \
--pipeline_config_path="/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn/pipeline.config" \
--checkpoint_dir="/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn"

In [ ]:
%load_ext tensorboard

In [ ]:
%tensorboard --logdir="/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/models/myft_ssd_resnet50_v1_fpn"

In [ ]:
PATH_TO_MODEL_DIR = "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/exported-models/saved_faster_rcnn_resnet50_v1"
PATH_TO_SAVED_MODEL = PATH_TO_MODEL_DIR + "/saved_model"
print(PATH_TO_SAVED_MODEL)

/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/exported-models/saved_faster_rcnn_resnet50_v1/saved_model


In [ ]:
import time

In [ ]:
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as viz_utils
from object_detection.utils import ops as utils_ops

%matplotlib inline

In [ ]:
print('Loading model...', end='')
start_time = time.time()

# Load saved model and build the detection function
detect_fn = tf.saved_model.load(PATH_TO_SAVED_MODEL)

end_time = time.time()
elapsed_time = end_time - start_time
print('Done! Took {} seconds'.format(elapsed_time))

Loading model...Done! Took 13.007854700088501 seconds


In [ ]:
PATH_TO_LABELS = '/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/annotations/label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
TEST_IMAGE_PATH = "/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/test/"

In [ ]:
IMAGE_PATHS = [TEST_IMAGE_PATH + f for f in os.listdir('/content/google_drive/MyDrive/tensorflow_OD_API_A/ssd_resnet50_v1_fpn/workspace/training_demo/images/test')
              if re.search(r'([a-zA-Z0-9\s_\\.\-\(\):])+(?i)(.jpg|.jpeg)$', f)] 

In [ ]:
IMAGE_PATHS_SELS = IMAGE_PATHS[1000:1005]

In [ ]:
def load_image_into_numpy_array(path):
    """Load an image from file into a numpy array.

    Puts image into numpy array to feed into tensorflow graph.
    Note that by convention we put it into a numpy array with shape
    (height, width, channels), where channels=3 for RGB.

    Args:
      path: the file path to the image

    Returns:
      uint8 numpy array with shape (img_height, img_width, 3)
    """
    return np.array(Image.open(path))

In [ ]:
for image_path in IMAGE_PATHS_SELS:

    print('Running inference for {}... '.format(image_path), end='')

    # image_path = TEST_IMAGE_PATH + image_path
    image_np = load_image_into_numpy_array(image_path)
    
    # Things to try:
    # Flip horizontally
    # image_np = np.fliplr(image_np).copy()

    # Convert image to grayscale
    # image_np = np.tile(
    #     np.mean(image_np, 2, keepdims=True), (1, 1, 3)).astype(np.uint8)

    # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
    input_tensor = tf.convert_to_tensor(image_np)
    # The model expects a batch of images, so add an axis with `tf.newaxis`.
    input_tensor = input_tensor[tf.newaxis, ...]

    # input_tensor = np.expand_dims(image_np, 0)
    detections = detect_fn(input_tensor)

    # All outputs are batches tensors.
    # Convert to numpy arrays, and take index [0] to remove the batch dimension.
    # We're only interested in the first num_detections.
    num_detections = int(detections.pop('num_detections'))
    detections = {key: value[0, :num_detections].numpy()
                   for key, value in detections.items()}
    detections['num_detections'] = num_detections

    # detection_classes should be ints.
    detections['detection_classes'] = detections['detection_classes'].astype(np.int64)

    image_np_with_detections = image_np.copy()

    viz_utils.visualize_boxes_and_labels_on_image_array(
          image_np_with_detections,
          detections['detection_boxes'],
          detections['detection_classes'],
          detections['detection_scores'],
          category_index,
          use_normalized_coordinates=True,
          max_boxes_to_draw=200,
          min_score_thresh=.30,
          agnostic_mode=False)

    plt.figure(figsize=(15,15))
    plt.imshow(image_np_with_detections)
    print('Done')

plt.show()


In [ ]:
!ps

    PID TTY          TIME CMD
      1 ?        00:00:00 docker-init
      7 ?        00:00:17 node
     11 ?        00:00:00 run.sh
     12 ?        00:00:01 kernel_manager_
     27 ?        00:00:01 tail
     39 ?        00:00:03 python3 <defunct>
     40 ?        00:00:14 colab-fileshim.
     67 ?        00:00:14 jupyter-noteboo
     68 ?        00:00:16 dap_multiplexer
     86 ?        00:02:02 python3
    106 ?        00:00:35 python3
    154 ?        00:00:00 bash
    155 ?        00:00:00 drive
    156 ?        00:00:00 grep
    222 ?        00:18:39 drive
    305 ?        00:00:00 bash
    306 ?        00:01:46 directoryprefet
    307 ?        00:00:00 tail
    308 ?        00:00:00 python3
   1419 ?        00:01:07 node
   4445 ?        00:00:03 tensorboard
   4457 ?        00:00:00 server
   4646 ?        00:00:00 ps


In [ ]:
!kill 4445 

In [ ]:
!mkdir "/content/google_drive/MyDrive/Tensorflow_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/models/myft_faster_rcnn_resnet50_v2"

In [ ]:
!cp "/content/google_drive/MyDrive/Tensorflow_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/models/myft_faster_rcnn_resnet50_v1/pipeline.config" \
"/content/google_drive/MyDrive/Tensorflow_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/models/myft_faster_rcnn_resnet50_v2"

In [ ]:
!python /content/models/research/object_detection/model_main_tf2.py \
--model_dir="/content/google_drive/MyDrive/Tensorflow_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/models/myft_faster_rcnn_resnet50_v2" \
--pipeline_config_path="/content/google_drive/MyDrive/Tensorflow_OD_API/faster_rcnn_resnet50_v1/workspace/training_demo/models/myft_faster_rcnn_resnet50_v2/pipeline.config"

2022-11-15 03:38:05.133150: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-15 03:38:05.850981: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-11-15 03:38:05.851082: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/lib64-nvidia
2022-11-15 03:38:05.851101: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed p